In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date

In [2]:
from requests import get
from bs4 import BeautifulSoup
import warnings
from time import sleep
from random import randint
from time import time
timestart_time = time()
warnings.warn('Warning Simulation')
from IPython.core.display import clear_output

/Users/viceproduction/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Warning Simulation
  


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [4]:
url='https://www.google.com/search?biw=1280&bih=666&tbs=cdr%3A1%2Ccd_min%3A1%2F31%2F2019%2Ccd_max%3A11%2F17%2F2019&tbm=nws&sxsrf=ACYBGNTglE7oldD0-y4p6YCgn2dqnP3X-A%3A1573979008321&ei=gAPRXfKZE4nXkwXyl5_YDA&q=%27%CE%A4.%CE%9F%CE%9C.%CE%A5.%27+%7C+%27%CE%A4%CE%9F.%CE%9C.%CE%A5.%27+%7C+%27%CE%A4%CE%9F%CE%9C%CE%A5%27+%7C+%27%CE%A4%CE%BF%CF%80%CE%B9%CE%BA%CE%AD%CF%82+%CE%9F%CE%BC%CE%AC%CE%B4%CE%B5%CF%82+%CE%A5%CE%B3%CE%B5%CE%AF%CE%B1%CF%82%27+%7C+%27%CE%A4%CE%BF%CF%80%CE%B9%CE%BA%CE%AD%CF%82+%CE%9C%CE%BF%CE%BD%CE%AC%CE%B4%CE%B5%CF%82+%CE%A5%CE%B3%CE%B5%CE%AF%CE%B1%CF%82%27&oq=%27%CE%A4.%CE%9F%CE%9C.%CE%A5.%27+%7C+%27%CE%A4%CE%9F.%CE%9C.%CE%A5.%27+%7C+%27%CE%A4%CE%9F%CE%9C%CE%A5%27+%7C+%27%CE%A4%CE%BF%CF%80%CE%B9%CE%BA%CE%AD%CF%82+%CE%9F%CE%BC%CE%AC%CE%B4%CE%B5%CF%82+%CE%A5%CE%B3%CE%B5%CE%AF%CE%B1%CF%82%27+%7C+%27%CE%A4%CE%BF%CF%80%CE%B9%CE%BA%CE%AD%CF%82+%CE%9C%CE%BF%CE%BD%CE%AC%CE%B4%CE%B5%CF%82+%CE%A5%CE%B3%CE%B5%CE%AF%CE%B1%CF%82%27&gs_l=psy-ab.4...17887.27045.0.28311.11.11.0.0.0.0.142.1138.1j8.9.0....0...1c.1j4.64.psy-ab..2.0.0....0.FYF4w5ECuLc'

In [5]:
#  Οι όροι αναζήτησης
# 'Τ.ΟΜ.Υ.' | 'ΤΟ.Μ.Υ.' | 'ΤΟΜΥ' | 'Τοπικές Ομάδες Υγείας' | 'Τοπικές Μονάδες Υγείας'

In [6]:
driver=webdriver.Chrome()
driver.get(url)

In [8]:
results_list=[]
pageCounter=0
numb_pages=24

while (pageCounter < numb_pages):
# We parse the content of each page
    doc = BeautifulSoup(driver.page_source, "html.parser")
    result_page=doc.find(id='search')
    news=result_page.findAll('div', attrs={'class':'g'})
    
    for new in news:
        main_items={}
        main_items['title']=new.h3.text
        main_items['description']=new.find('div',attrs={'class':'st'}).text
        main_items['media']=new.findAll('span')[0].text
        main_items['date']=new.findAll('span')[2].text
        main_items['href']=new.a.get('href')

        results_list.append(main_items)
        if new.find('div',attrs={'class':'YiHbdc card-section'}) is not None:
            sub_news=new.findAll('div',attrs={'class':'YiHbdc card-section'})
            for sub in sub_news:
                items={}
                items['title']=sub.a.text
                items['description']='Sub news - No description'
                items['media']=sub.span.text
                items['date']=sub.findAll('span')[2].text
                items['href']=sub.a.get('href')
                results_list.append(items)
    pageCounter+=1
    driver.find_element_by_id('pnnext').click()
    
    print(pageCounter)
    clear_output(wait=True)

24


In [9]:
pd.DataFrame(results_list).to_csv('TOMY.CSV',index=False, encoding='utf-8')

In [10]:
df=pd.read_csv('TOMY.CSV')

In [11]:
from dateutil.parser import parse

In [12]:
def parse_date(date):
    return(parse(date))

In [17]:
df['parsed_date']=pd.to_datetime(df['date'].apply(lambda x: parse_date(x)))

In [18]:
df['month']=df.parsed_date.dt.month

In [19]:
df.sort_values(by='parsed_date').to_csv('TOMY_with_dates.CSV',index=False, encoding='utf-8')